In [1]:
import numpy as np
from pyLAMMPS import LAMMPS_setup
from pyLAMMPS.tools.general_utils import work_json
from pyLAMMPS.analysis.mechanical_properties import get_stiffness_tensor, compute_VRH

## Mechanical properties

This notebook uses pyLAMMPS to setup simulations to compute mechanical properties using finite differences

In [2]:
lammps_setup = LAMMPS_setup( system_setup = "input/setup_mechanical.yaml", 
                             simulation_default = "input/defaults.yaml",
                             simulation_ensemble = "input/ensemble.yaml",
                             simulation_sampling = "input/sampling_mechanical.yaml",
                             submission_command = "qsub"
                            )

## Setting up equilibration

In this section the inital system is equilibrated at desired temperature and pressure.

1) Setup simulation


In [5]:
# Define the simulation folder
simulation_folder = "equilibration"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = [ "em", "npt" ] 

# Define the simulation time per ensemble in nano seconds (for em the number of iterations is provided in the ensemble yaml)
simulation_times = [ 0, 0.1 ]

# Define initial data file (for each pressure & temperature state one)
initial_systems = [ "/home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/input/ettringite_bulk.data" ]

# Define force field file
ff_file = "/home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/input/ettringite_bulk.params"

# Provide kwargs that should be passed into the input template directly
input_kwargs = {  }

# Define number of copies
copies = 2

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set    = 0

lammps_setup.prepare_simulation( folder_name = simulation_folder, ensembles = ensembles, simulation_times = simulation_times,
                                 initial_systems = initial_systems, input_kwargs = input_kwargs, copies = copies,
                                 ff_file = ff_file, off_set = off_set )


Intial system provided for at: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/input/ettringite_bulk.data



2) Submit jobs to cluster

In [6]:
# Submit the simulations
lammps_setup.submit_simulation()


Submitting simulations at Temperature = 298 K, Pressure = 1 bar

Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_0/job_298_1.sh
11364914


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_1/job_298_1.sh
11364915


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_2/job_298_1.sh
11364916




## Setting up deformation

In this section the equilibrated system is deformed in each direction and the corresponding pressure tensor is sampled

1) Setup simulation


In [9]:
# Define deformation directions
deformation_directions = [ "xx", "yy", "zz", "xy", "xz", "yz", "undeformed" ]

# Define deformation rates
deformation_rates = [ -0.02, -0.01, 0.00, 0.01, 0.02 ]

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = [ "em", "nvt", "nvt" ]

# Define the simulation time per ensemble in nano seconds (for em the number of iterations is provided in the ensemble yaml)
simulation_times = [ 0.0, 0.1, 0.1 ]

# Define initial data file (for each pressure & temperature state one)
initial_systems = [ "/home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_0/01_npt/npt.restart" ]

# Define force field file
ff_file = "/home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/input/ettringite_bulk.params"

# Define number of copies
copies = 2

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set    = 0

job_files = [ [] for _ in lammps_setup.system_setup["temperature"] ]

for deformation_direction in deformation_directions:

    for deformation_rate in deformation_rates:
        # Don't simulate the undeformed system in each deformation direction, as this would simulate the undeformed system 6*no_copies
        if (deformation_rate == 0.0 and not deformation_direction == "undeformed") or \
           (deformation_rate != 0.0 and deformation_direction == "undeformed"):
            continue
        else:
            # Define the simulation folder
            simulation_folder = f"deformation/{deformation_direction}/{deformation_rate}"

            # Provide kwargs that should be passed into the input template directly
            input_kwargs = { "deformation": { "direction": deformation_direction,
                                              "rate": deformation_rate } }


            lammps_setup.prepare_simulation( folder_name = simulation_folder, 
                                             ensembles = ensembles, 
                                             simulation_times = simulation_times,
                                             initial_systems = initial_systems, 
                                             input_kwargs = input_kwargs, 
                                             copies = copies,
                                             ff_file = ff_file, 
                                             off_set = off_set )
            
            for j, files in enumerate( lammps_setup.job_files ):
                job_files[j].extend( files )

lammps_setup.job_files = job_files


Intial system provided for at: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_0/01_npt/npt.restart

Restart file is provided. Continue simulation from there!


Intial system provided for at: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_0/01_npt/npt.restart

Restart file is provided. Continue simulation from there!


Intial system provided for at: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_0/01_npt/npt.restart

Restart file is provided. Continue simulation from there!


Intial system provided for at: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_0/01_npt/npt.restart

Restart file is provided. Continue simulation from there!


Intial 

2) Submit jobs to cluster

In [11]:
# Submit the simulations
lammps_setup.submit_simulation()


Submitting simulations at Temperature = 298 K, Pressure = 1 bar

Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/deformation/xx/-0.02/temp_298.1_pres_1.0/copy_0/job_298_1.sh
11364936


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/deformation/xx/-0.02/temp_298.1_pres_1.0/copy_1/job_298_1.sh
11364937


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/deformation/xx/-0.02/temp_298.1_pres_1.0/copy_2/job_298_1.sh
11364938


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/deformation/xx/-0.01/temp_298.1_pres_1.0/copy_0/job_298_1.sh
11364939


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/deformation/xx/-0.01/temp_298.1_pres_1.0/copy_1/job_298_1.sh
11364940


Submitting job

## Analysis

### Analyze lattice properties amd density from equilibration

In [13]:
# Define the analysis folder
analysis_folder = "equilibration"

# Define analysis ensemble
ensemble = "01_npt"  

# Properties to extract
properties = [ "a", "b", "c", "alpha", "beta", "gamma" ]

# Suffix of output file
output_suffix = "lattice"

# Percentage to discard from beginning of the simulation
fraction = 0.0

# Extract properties from LAMMPS and analyse them
lammps_setup.analysis_extract_properties( analysis_folder = analysis_folder, 
                                          ensemble = ensemble, 
                                          extracted_properties = properties, 
                                          output_suffix = output_suffix, 
                                          fraction = fraction )

# Properties to extract
properties = [ "density" ]

# Suffix of output file
output_suffix = "density"

# Percentage to discard from beginning of the simulation
fraction = 0.0

# Extract properties from LAMMPS and analyse them
lammps_setup.analysis_extract_properties( analysis_folder = analysis_folder, 
                                          ensemble = ensemble, 
                                          extracted_properties = properties, 
                                          output_suffix = output_suffix, 
                                          fraction = fraction )

Temperature: 298.15, Pressure: 1.0
   /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_0/01_npt/npt.lattice
   /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_1/01_npt/npt.lattice
   /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_2/01_npt/npt.lattice


Averaged values over all copies:

   property        mean       std unit
0        a   33.677120  0.022423   AA
1        b   33.677120  0.022423   AA
2        c   42.829168  0.028513   AA
3    alpha   90.000000  0.000000  deg
4     beta   90.000000  0.000000  deg
5    gamma  120.000000  0.000000  deg 

Temperature: 298.15, Pressure: 1.0
   /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/mechanical_properties/ettringite/equilibration/temp_298.1_pres_1.0/copy_0/01_npt/n

### Analyze pressure tensor from production

In [ ]:
# Define analysis ensemble
ensemble = "02_nvt"  

# Properties to extract
properties = [ "pxx", "pyy", "pzz", "pxy", "pxz", "pyz" ]

# Suffix of output file
output_suffix = "pressure"

# Percentage to discard from beginning of the simulation
fraction = 0.0

# Create resulting dictionary
results = {}

for deformation_direction in deformation_directions:

    results[deformation_direction] = {}

    for deformation_rate in deformation_rates:

        if (deformation_rate == 0.0 and not deformation_direction == "undeformed") or \
           (deformation_rate != 0.0 and deformation_direction == "undeformed"):
            continue
        else:
            # Define the analysis folder
            analysis_folder = f"deformation/{deformation_direction}/{deformation_rate}"

            # Extract properties from LAMMPS and analyse them
            lammps_setup.analysis_extract_properties( analysis_folder = analysis_folder, 
                                                      ensemble = ensemble, 
                                                      extracted_properties = properties, 
                                                      output_suffix = output_suffix, 
                                                      fraction = fraction )
            

            # Add each copy to the results dict
            sim_results = work_json( f"{analysis_folder}/results.json" )[ensemble]["data"]
            results[deformation_direction][deformation_rate] = { item for key,item in sim_results.items() if key != "average" }


In [ ]:
# This deformation/pressure direction order is used to evaluate the stiffnes tensor
directions = [ "xx", "yy", "zz", "yz", "xz", "xy" ]

# Save final results
final_results = { }

for copy in range( copies ):
    # A 3D list representing the pressure values for each deformation direction.
    # The innermost list contains the pressure values for one deformation magnitude in the following order: [pxx, pyy, pzz, pyz, pxz, pxy]. 
    # The middle list contains the pressure values for each deformation magnitude. 
    # The outermost list contains the pressure values for each deformation magnitude in each deformation direction (xx,yy,zz,yz,xz,xy).
    pressure = [ ]

    for deformation_direction in directions:
        dummy = []
        for deformation_rate in deformation_rates:
            if deformation_rate == 0.0:
                res = results["undeformed"][deformation_rate][f"copy_{copy}"]
            else:
                res = results[deformation_direction][deformation_rate][f"copy_{copy}"]
                
            # Add the mean pressure values of each direction (conversion from atm in GPa)
            dummy.append( [ res[f"p{d}"]["mean"]* 101325 / 1e9 for d in directions ] )
        pressure.append( dummy )

    ## Get slope of stress-strain curve at low deformation 

    stiffness_tensor = get_stiffness_tensor( pressure = pressure, 
                                             deformation_magnitudes = deformation_rates )
    
    ## Computation of mechanical properties 

    #Voigt Reuss Hill
    K, G, E, nu = compute_VRH( stiffness_tensor )

    final_results[f"copy_{copy}"] = { "C": { "mean": stiffness_tensor.tolist() },
                                      "K": { "mean": K },
                                      "G": { "mean": G },
                                      "nu": { "mean": nu } 
                                    }
    
final_results["average"] = { key: { "mean": np.mean( [ copy_data[key]["mean"] for copy_data in final_results.values() ] ).tolist(),
                                    "std": np.std( [ copy_data[key]["mean"] for copy_data in final_results.values() ], ddof=1 ).tolist() }  
                                    for key in ["C","K","G","nu"] }

work_json( file_path = f"{lammps_setup.system_setup['folder']}/{lammps_setup.system_setup['sytem']}/deformation/final_results.json",
           data =  final_results,
           to_do = "write" )